In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# from google.colab import drive
# drive.mount("/content/MyDrive/")

In [ ]:
import os
import sys
from pathlib import Path
import argparse
import torch

sys.path.append("/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/CF_HV-AV_2/GAIL/") # used only in colab
from buffer import SerializedBuffer
from utils import evaluation, visualize
from agent import ALGOS, EXP_ALGOS
from trainer import Trainer

sys.path.append("/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/") # used only in colab
sys.path.append(str(Path(os.getcwd()).parent.absolute())) # used only in ipython
sys.path.append(str(Path(os.getcwd()).parent.parent.absolute())) # used only in ipython
from CF_Env import CF_Env

# begin training

In [ ]:
def run(args):
    # train Imitation Learning algorithms
    env = CF_Env()

    buffer_exp = SerializedBuffer(
        path=args.buffer,
        device=torch.device("cuda" if args.cuda else "cpu")
    )

    algo = ALGOS[args.algo](
        buffer_exp=buffer_exp,
        state_shape=env.observation_space.shape,
        action_shape=env.action_space.shape,
        device=torch.device("cuda" if args.cuda else "cpu"),
        seed=args.seed,
        rollout_length=args.rollout_length
    )

    # path to save logs
    log_dir = os.path.join('logs', args.algo)

    trainer = Trainer(
        env=env,
        algo=algo,
        log_dir=log_dir,
        buffer_path=args.buffer,
        num_steps=args.num_steps,
        valid_path=args.valid_path
    )
    trainer.train()

In [ ]:
# if __name__ == '__main__':
#     p = argparse.ArgumentParser()
#
#     """used only in command line"""
#     p.add_argument('--buffer', type=str, help='path to the demonstration buffer',
#                    default=str(Path(os.getcwd()).parent.absolute())+'/'+'train_cf_4.npy')
#     """used only in colab"""
#     # p.add_argument('--buffer', type=str, help='path to the demonstration buffer',
#     #                default="/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/CF_HV-AV_4/train_cf_4.npy")
#
#     """used only in command line"""
#     p.add_argument('--valid_path', type=str, help='path to the validation buffer',
#                    default=str(Path(os.getcwd()).parent.absolute())+'/'+'test_cf_4.npy')
#     """used only in colab"""
#     # p.add_argument('--valid_path', type=str, help='path to the validation buffer',
#     #                default="/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/CF_HV-AV_4/test_cf_4.npy")
#
#     p.add_argument('--rollout_length', type=int, help='rollout length of the buffer',
#                    default=25)
#     p.add_argument('--num_steps', type=int, help='number of steps to train',
#                    default=60000)
#     p.add_argument('--algo', type=str, help='Imitation Learning algorithm to be trained',
#                    default='gail')
#     p.add_argument('--cuda', action='store_true')
#     p.add_argument('--seed', type=int, help='random seed',
#                    default=0)
#     p.add_argument('-f')
#     args = p.parse_args()
#
#     run(args)

# begin testing

In [ ]:
def run(args):
    env = CF_Env()
    agent = EXP_ALGOS[args.algo](env.observation_space.shape,
                                 env.action_space.shape,
                                 device=torch.device("cuda" if args.cuda else "cpu"),
                                 path=args.policy+"/"+args.algo+"/model/"
                                 )

    evaluation(name=args.algo,
               env=env,
               algo=agent,
               buffer=args.buffer,
               seed=args.seed
               )

    visualize(name=args.algo,
              buffer=args.buffer,
              traj_path=args.traj_path,
              num=args.num)


In [ ]:
if __name__ == '__main__':
    p = argparse.ArgumentParser()
    
    """used only in command line"""
    p.add_argument('--buffer', type=str, help='path to the demonstration buffer',
                   default=str(Path(os.getcwd()).parent.absolute())+'/'+'test_cf_4.npy')
    """used only in colab"""
    # p.add_argument('--buffer', type=str, help='path to the demonstration buffer',
    #                default="/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/CF_HV-AV_2/test_cf_4.npy")
    
    p.add_argument('--policy', type=str, help='path to the trained policy',
                   default='logs/')
    p.add_argument('--traj_path', type=str, help='path to the simulated trajectory',
                   default='result/')
    p.add_argument('--num', type=int, help='index of trajectory to be visualized',
                   default=3)
    p.add_argument('--algo', type=str, help='Imitation Learning algorithm to be trained',
                   default='gail')
    p.add_argument('--cuda', action='store_true')
    p.add_argument('--seed', type=int, help='random seed',
                   default=0)
    p.add_argument('-f')
    args = p.parse_args()

    run(args)